# 标注和预测

```{note}
要对锚框进行训练，需使用真实边界框标注锚框，包括锚框的类别和偏移。<br/>
预测时，需根据锚框的类别和偏移反向推导出边界框，并进行过滤。
```

## 交并比

如何衡量锚框和真实边界框的相似性呢？我们可以使用Jaccard系数，即给定集合A和B，用它们交集的大小除以并集的大小：

$$
\mathrm{J}(A, B) = \frac{|A\cap{B}|}{|A\cup{B}|}
$$

对于两个边界框，就是两个边界框相交面积和相并面积之比，即交并比（intersection over union，IoU）。

![jupyter](../images/e/iou.svg)

## 标注类别

假设锚框是 $A_{1},...,A_{n}$，真实边界框是 $B_{1},...,B_{m}$，其中 $n\ge{m}$。

我们定义一个矩阵 $\mathbf{X}\in\mathbb{R}^{n\times{m}}$，其中第 $i$ 行第 $j$ 列的元素 $x_{ij}$ 为锚框 $A_{i}$ 和边界框 $B_{j}$ 的IoU。

那么分配包含以下步骤：

1. 找到矩阵 $\mathbf{X}$ 的最大元素，设在第 $i_{1}$ 行第 $j_{1}$ 列，那么把 $B_{j_{1}}$ 分配给 $A_{i_{1}}$，并丢弃地第 $i_{1}$ 行和第 $j_{1}$ 列的所有元素。

2. 在 $\mathbf{X}$ 的剩余元素中找最大元素，设在第 $i_{2}$ 行第 $j_{2}$ 列，那么类似地分配然后丢弃。

3. 直到我们丢弃完 $\mathbf{X}$ 的所有元素，这时每一个边界框都一个锚框与之对应。

4. 遍历剩下的 $n - m$ 个锚框，对于每一个锚框找到与其IoU最大的边界框，当此IoU大于阈值时进行分配。

下图是 $x_{23}\to x_{71}\to x_{54}\to x_{92}$

![jupyter](../images/e/anchor-label.svg)

## 计算偏移

假设一个锚框 $A$ 被分配了一个真实边界框 $B$，我们还需要根据 $A$ 和 $B$ 的坐标计算偏移。

记 $A$ 和 $B$ 的中心坐标分别为 $(x_{a},y_{a})$ 和 $(x_{b},y_{b})$，宽度分别为 $w_{a}$ 和 $w_{b}$，高度分别为 $h_{a}$ 和 $h_{b}$，那么我们可以将 $A$ 的偏移量标记为：

$$\left(\frac{x_{b}-x_{a}}{w_{a}}, \frac{y_{b}-y_{a}}{h_{a}}, \log{\frac{w_b}{w_a}}, \log{\frac{h_b}{h_a}}\right)$$


## 预测

在预测期间，我们先为图像生成多个锚框，再为这些锚框一一预测类别和偏移量。

假设有锚框 $A$，其坐标为 $(x_{a}, y_{a})$，宽高分别为 $w_{a}$ 和 $h_{a}$，预测的偏移为 $(f_{1}, f_{2}, f_{3}, f_{4})$，那么预测边界框为：

$$
\begin{equation}
\begin{split}
x_{b} =& x_{a} + f_{1}w_{a} \\
y_{b} =& y_{a} + f_{2}h_{a} \\
w_{b} =& w_{a}\exp{(f_{3})} \\
h_{b} =& h_{a}\exp{(f_{4})}
\end{split}
\end{equation}
$$

当有许多锚框时，预测边界框可能会明显重叠，我们可以使用非极大值抑制 (non-maximum suppression，NMS)合并相似的预测边界框。

对于一个预测边界框 $B$ ，目标检测模型会计算每个类的预测概率，假设最大的预测概率为 $p$，则该概率所对应的类别 $B$ 即为预测的类别，称 $p$ 为 $B$ 的置信度。

NMS的具体步骤为：

1. 选取置信度最高的预测边界框 $B_{1}$，移除与 $B_{1}$ 的IoU 超过预定阈值 $\epsilon$ 的边界框

2. 从剩余边界框中选取置信度第二高的预测边界框 $B_{2}$...

3. 重复上述过程，直至没有边界框

## 多尺度目标检测

若为每个像素生成锚框，那么锚框数量就太多了。

我们可以对像素进行均匀采样，且让面积小的锚框采样次数多一些，面积大的锚框采样次数少一些。

![jupyter](../images/e/multiscale1.svg)

![jupyter](../images/e/multiscale2.svg)